In [1]:
# Import libraries
# transformers: Provides access to pre-trained models like CLAP.
from transformers import ClapModel, AutoProcessor

# librosa: Used for audio analysis and loading audio files.
import librosa

# os: Provides a way to interact with the operating system, like navigating directories.
import os

import torch
import pandas as pd
import numpy as np

In [2]:
# Load CLAP model + processor
model = ClapModel.from_pretrained("laion/clap-htsat-unfused")
processor = AutoProcessor.from_pretrained("laion/clap-htsat-unfused")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/614M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [3]:
#Small test to print all the parameters of CLAP
print('total number of CLAP\'s parameters:', sum(p.numel() for p in model.parameters()))

total number of CLAP's parameters: 153492890


In [4]:
#Calculating the size of the data of all the parameters
param_size = 0 # We are using a for loop to add size of all parameters os initialised as 0 at the beginning

for param in model.parameters():
    #In increments we are adding the size of each parameter given by formula
    #param.nelement() is giving total number of elements in each parameter
    #param.element_size() is giving size of element in bytes
    param_size += param.nelement() * param.element_size()

#Buffers are tensors but not learnable parameters but still take up space
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

#Total size adds buffer size and parameter size
#Converted into MB
size_all_mb = (param_size + buffer_size) / 1024**2
print('CLAP model size: {:.3f}MB'.format(size_all_mb))

CLAP model size: 585.912MB


In [5]:
#This section is to convert the uploaded zip files into folders
import zipfile

# Define the paths to your zip files
zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

# Define the directory where you want to extract the files
extract_dir = "/content/" # You can change this if you want to extract elsewhere

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract each zip file
for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Erro r: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


# Process audio

In [6]:
audio_stimuli = []
stimuli_path = "/content/Exp1/Stimuli/"  #/content/Exp2/Stimuli/ if we are running experiment 2

#This section adds all the audio waveforms into the list audio_stimuli
for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        #Clap has already been trained on a sample rate of 48,000 so we should use what it knows already
        print(wav_path)
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

/content/Exp1/Stimuli/01_B_CTu_1.wav
/content/Exp1/Stimuli/02_B_CTu_2.wav
/content/Exp1/Stimuli/03_B_Tu_3.wav
/content/Exp1/Stimuli/04_B_Tu_4.wav
/content/Exp1/Stimuli/05_B_CTb_1.wav
/content/Exp1/Stimuli/06_B_CTb_2.wav
/content/Exp1/Stimuli/07_B_BTb_3.wav
/content/Exp1/Stimuli/08_B_TTb_4.wav
/content/Exp1/Stimuli/09_B_Ho_3.wav
/content/Exp1/Stimuli/10_B_Ho_4.wav
/content/Exp1/Stimuli/11_B_Ho_5.wav
/content/Exp1/Stimuli/12_B_Tr_4.wav
/content/Exp1/Stimuli/13_B_PTr_5.wav
/content/Exp1/Stimuli/14_B_PTr_6.wav
/content/Exp1/Stimuli/15_W_CBa_1.wav
/content/Exp1/Stimuli/16_W_CBa_2.wav
/content/Exp1/Stimuli/17_W_Ba_3.wav
/content/Exp1/Stimuli/18_W_Ba_4.wav
/content/Exp1/Stimuli/19_W_BCl_2.wav
/content/Exp1/Stimuli/20_W_BCl_3.wav
/content/Exp1/Stimuli/21_W_ClB_4.wav
/content/Exp1/Stimuli/22_W_ClB_5.wav
/content/Exp1/Stimuli/23_W_ClB_6.wav
/content/Exp1/Stimuli/24_W_Ob_4.wav
/content/Exp1/Stimuli/25_W_Ob_5.wav
/content/Exp1/Stimuli/26_W_EH_4.wav
/content/Exp1/Stimuli/27_W_EH_5.wav
/content/Exp1

In [7]:
#verifcation if we opened correct file
print(audio_stimuli)

[array([-2.6257991e-05, -1.8168332e-05,  4.3254666e-05, ...,
       -1.1775010e-05,  1.0805696e-06, -3.3297738e-08], dtype=float32), array([-2.1216942e-05, -4.7350541e-06,  1.5635709e-05, ...,
        2.8016274e-08, -1.7804325e-08,  6.9179218e-09], dtype=float32), array([-9.8106102e-06,  1.1119668e-05, -3.5152134e-05, ...,
       -3.4414730e-05, -1.3167290e-05,  4.1392400e-06], dtype=float32), array([-2.9096404e-06, -1.3023089e-05, -4.7573376e-06, ...,
       -3.4261415e-05, -2.2727640e-05,  2.1467149e-06], dtype=float32), array([ 2.7754413e-06, -7.0806864e-06,  1.1189784e-05, ...,
       -1.2227122e-06,  7.4793968e-07, -2.8134855e-07], dtype=float32), array([-6.7172846e-07,  5.4504471e-06, -5.0830917e-05, ...,
       -1.6670765e-06,  1.9083664e-06,  0.0000000e+00], dtype=float32), array([ 1.6268901e-05, -3.3643319e-05,  2.5531041e-05, ...,
       -3.3158278e-05, -2.5886366e-05,  7.7379474e-07], dtype=float32), array([ 6.1509754e-06,  2.7266901e-06, -4.9879025e-05, ...,
       -1.23645

In [8]:
#Processes the audio waveform in the audio_stimuli list
#The process involves converting the audio files into tensors (As PyTorch deals with Tensors)
inputs = processor(audios=audio_stimuli, return_tensors="pt", padding=True, sampling_rate=48000)

#Inputs the tensors into the CLAP model to extract the audio embeddings
#These embeddings are numerical representations
audio_embeddings = model.get_audio_features(**inputs)

In [9]:
#Returning dimensions of the tensors
#The 59 is how many audio files there are
#The 512 is the size of any audio embedding
print(audio_embeddings.shape)

torch.Size([59, 512])


# Process text

In [10]:
#This is generating the prompts for each section
#We have 2 sections of discrete and dimensional
#Each section has 2 subsections of perceived and induced
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]

discrete_captions_perceived = ["I perceive this sound as " + tag for tag in discrete_tags]
print(discrete_captions_perceived)
discrete_captions_induced = ["This sound makes me feel " + tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake"]

dimensional_captions_perceived = ["I perceive this sound as " + tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = ["This sound makes me feel " + tag for tag in dimensional_tags]
print(dimensional_captions_induced)


['I perceive this sound as happiness', 'I perceive this sound as sadness', 'I perceive this sound as anger', 'I perceive this sound as tenderness', 'I perceive this sound as fear']
['This sound makes me feel happiness', 'This sound makes me feel sadness', 'This sound makes me feel anger', 'This sound makes me feel tenderness', 'This sound makes me feel fear']
['I perceive this sound as positive', 'I perceive this sound as relaxed', 'I perceive this sound as awake']
['This sound makes me feel positive', 'This sound makes me feel relaxed', 'This sound makes me feel awake']


In [11]:
#Combines all the generated text prompts from previous cell into one list
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

# NOTE: currently using only dimensional_captions_induced
tag_inputs = processor(text=dimensional_captions_induced, return_tensors="pt", padding=True)
tag_embeds = model.get_text_features(**tag_inputs)

# Generate outputs

In [12]:
#Computes the similaity matrix between each audio clip and text tag
#Computes this using dot product between the audio embeddings and text embeddings
sims = torch.matmul(audio_embeddings, tag_embeds.T)
print(sims.shape)

torch.Size([59, 3])


## Load csv files and extract related columns

In [13]:
IDim_positive_path = '/content/Exp1/Data/IDim'
IDim_positive_response_dfs = []

for file in os.listdir(IDim_positive_path):
    if file.endswith(".csv"):
        file_path = os.path.join(IDim_positive_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['positive']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDim_positive_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDim_positive_response_dfs:
    master_human_responses_df_positive = pd.concat(IDim_positive_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_positive.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_positive.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_positive_path.")


Master human responses DataFrame shape: (3835, 1)

Master human responses (first 5 rows):
   positive
0      5.24
1      4.78
2      6.04
3      3.36
4      4.94



In [14]:
IDim_relaxed_path = '/content/Exp1/Data/IDim'
IDim_relaxed_response_dfs = []

for file in os.listdir(IDim_relaxed_path):
    if file.endswith(".csv"):
        file_path = os.path.join(IDim_relaxed_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['relaxed']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDim_relaxed_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDim_relaxed_response_dfs:
    master_human_responses_df_relaxed = pd.concat(IDim_relaxed_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_relaxed.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_relaxed.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_relaxed_path.")


Master human responses DataFrame shape: (3835, 1)

Master human responses (first 5 rows):
   relaxed
0     5.00
1     4.21
2     6.35
3     5.67
4     4.55



In [15]:
IDim_awake_path = '/content/Exp1/Data/IDim'
IDim_awake_response_dfs = []

for file in os.listdir(IDim_awake_path):
    if file.endswith(".csv"):
        file_path = os.path.join(IDim_awake_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['awake']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDim_awake_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDim_awake_response_dfs:
    master_human_responses_df_awake = pd.concat(IDim_awake_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_awake.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_awake.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_awake_path.")


Master human responses DataFrame shape: (3835, 1)

Master human responses (first 5 rows):
   awake
0   5.60
1   4.79
2   6.34
3   4.26
4   5.01



# Prepare features X and targets y

In [16]:
from sklearn.model_selection import train_test_split


num_participants_positive = len(IDim_positive_response_dfs)
if master_human_responses_df_positive.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list_positive = []
for _ in range(num_participants_positive):
    X_list_positive.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_positive = np.array(X_list_positive)

# Extract y from the concatenated DataFrame
y_positive = master_human_responses_df_positive[['positive']].values

print(f"Shape of X (features) after implicit alignment: {X_positive.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_positive.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_positive.shape[0] != y_positive.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---

TEST_PERCENTAGE_positive = 0.2  # Adjust as needed
TRAIN_PERCENTAGE_positive = 1 - TEST_PERCENTAGE_positive  # Adjust as needed

X_train_positive, X_test_positive, y_train_positive, y_test_positive = train_test_split(
    X_positive, y_positive, test_size=TEST_PERCENTAGE_positive, random_state=42
)

print(f"Training set size (X_train_positive, y_train_positive): {X_train_positive.shape}, {y_train_positive.shape}")
print(f"Testing set size (X_test_positive, y_test_positive): {X_test_positive.shape}, {y_test_positive.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 1)

Training set size (X_train_positive, y_train_positive): (3068, 512), (3068, 1)
Testing set size (X_test_positive, y_test_positive): (767, 512), (767, 1)



In [17]:
from sklearn.model_selection import train_test_split


num_participants_relaxed = len(IDim_relaxed_response_dfs)
if master_human_responses_df_relaxed.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list_relaxed = []
for _ in range(num_participants_relaxed):
    X_list_relaxed.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_relaxed = np.array(X_list_relaxed)

# Extract y from the concatenated DataFrame
y_relaxed = master_human_responses_df_relaxed[['relaxed']].values

print(f"Shape of X (features) after implicit alignment: {X_relaxed.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_relaxed.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_relaxed.shape[0] != y_relaxed.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---

TEST_PERCENTAGE_relaxed = 0.2  # Adjust as needed
TRAIN_PERCENTAGE_relaxed = 1 - TEST_PERCENTAGE_relaxed  # Adjust as needed

X_train_relaxed, X_test_relaxed, y_train_relaxed, y_test_relaxed = train_test_split(
    X_relaxed, y_relaxed, test_size=TEST_PERCENTAGE_relaxed, random_state=42
)

print(f"Training set size (X_train_relaxed, y_train_relaxed): {X_train_relaxed.shape}, {y_train_relaxed.shape}")
print(f"Testing set size (X_test_relaxed, y_test_relaxed): {X_test_relaxed.shape}, {y_test_relaxed.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 1)

Training set size (X_train_relaxed, y_train_relaxed): (3068, 512), (3068, 1)
Testing set size (X_test_relaxed, y_test_relaxed): (767, 512), (767, 1)



In [19]:
from sklearn.model_selection import train_test_split


num_participants_awake = len(IDim_awake_response_dfs)
if master_human_responses_df_awake.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list_awake = []
for _ in range(num_participants_awake):
    X_list_awake.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_awake = np.array(X_list_awake)

# Extract y from the concatenated DataFrame
y_awake = master_human_responses_df_awake[['awake']].values

print(f"Shape of X (features) after implicit alignment: {X_awake.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_awake.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_awake.shape[0] != y_awake.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---

TEST_PERCENTAGE_awake = 0.2  # Adjust as needed
TRAIN_PERCENTAGE_awake = 1 - TEST_PERCENTAGE_awake  # Adjust as needed

X_train_awake, X_test_awake, y_train_awake, y_test_awake = train_test_split(
    X_awake, y_awake, test_size=TEST_PERCENTAGE_awake, random_state=42
)

print(f"Training set size (X_train_awake, y_train_awake): {X_train_awake.shape}, {y_train_awake.shape}")
print(f"Testing set size (X_test_awake, y_test_awake): {X_test_awake.shape}, {y_test_awake.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 1)

Training set size (X_train_awake, y_train_awake): (3068, 512), (3068, 1)
Testing set size (X_test_awake, y_test_awake): (767, 512), (767, 1)



# Train regression head (=MLP, a few projection layers)

In [20]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_positive = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42, #Determines random number generation for weights and bias
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor_positive training...")
mlp_regressor_positive.fit(X_train_positive, y_train_positive)
print("\nMLP Regressor_positive training complete.")


Starting MLP Regressor_positive training...
Iteration 1, loss = 12.72261545
Validation score: -4.702067
Iteration 2, loss = 7.70512113
Validation score: -1.463984
Iteration 3, loss = 2.84515621
Validation score: -0.061137


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 4, loss = 1.93240131
Validation score: 0.178245
Iteration 5, loss = 1.58192796
Validation score: 0.193602
Iteration 6, loss = 1.51326816
Validation score: 0.236566
Iteration 7, loss = 1.47760809
Validation score: 0.241435
Iteration 8, loss = 1.44908837
Validation score: 0.249871
Iteration 9, loss = 1.43150253
Validation score: 0.255931
Iteration 10, loss = 1.41774539
Validation score: 0.261044
Iteration 11, loss = 1.40722722
Validation score: 0.265966
Iteration 12, loss = 1.40098059
Validation score: 0.268688
Iteration 13, loss = 1.39731622
Validation score: 0.273675
Iteration 14, loss = 1.39158961
Validation score: 0.275873
Iteration 15, loss = 1.38729454
Validation score: 0.280165
Iteration 16, loss = 1.38461277
Validation score: 0.282236
Iteration 17, loss = 1.38093393
Validation score: 0.279980
Iteration 18, loss = 1.37861638
Validation score: 0.285789
Iteration 19, loss = 1.37541855
Validation score: 0.283954
Iteration 20, loss = 1.37275084
Validation score: 0.286820
Ite

In [21]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_relaxed = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42, #Determines random number generation for weights and bias
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor_relaxed training...")
mlp_regressor_relaxed.fit(X_train_relaxed, y_train_relaxed)
print("\nMLP Regressor_relaxed training complete.")


Starting MLP Regressor_relaxed training...
Iteration 1, loss = 11.83117368
Validation score: -3.637644
Iteration 2, loss = 7.07593221
Validation score: -0.919648
Iteration 3, loss = 2.72778035
Validation score: -0.037343
Iteration 4, loss = 1.97416178
Validation score: 0.218441


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 5, loss = 1.63037506
Validation score: 0.257778
Iteration 6, loss = 1.55200456
Validation score: 0.288123
Iteration 7, loss = 1.50841260
Validation score: 0.306247
Iteration 8, loss = 1.47718732
Validation score: 0.318363
Iteration 9, loss = 1.45722948
Validation score: 0.326982
Iteration 10, loss = 1.44293574
Validation score: 0.332684
Iteration 11, loss = 1.43283491
Validation score: 0.337708
Iteration 12, loss = 1.42584631
Validation score: 0.341208
Iteration 13, loss = 1.42240338
Validation score: 0.343087
Iteration 14, loss = 1.41797276
Validation score: 0.345478
Iteration 15, loss = 1.41325286
Validation score: 0.346534
Iteration 16, loss = 1.41387601
Validation score: 0.345977
Iteration 17, loss = 1.41106987
Validation score: 0.348058
Iteration 18, loss = 1.40486447
Validation score: 0.349565
Iteration 19, loss = 1.40129718
Validation score: 0.351324
Iteration 20, loss = 1.40137287
Validation score: 0.351722
Iteration 21, loss = 1.40196132
Validation score: 0.349002
It

In [22]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_awake = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42, #Determines random number generation for weights and bias
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor_awake training...")
mlp_regressor_awake.fit(X_train_awake, y_train_awake)
print("\nMLP Regressor_awake training complete.")


Starting MLP Regressor_awake training...
Iteration 1, loss = 16.63529883
Validation score: -7.492959
Iteration 2, loss = 10.47552155
Validation score: -2.779072
Iteration 3, loss = 3.58510799
Validation score: -0.107331
Iteration 4, loss = 2.09506068
Validation score: -0.014619


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 5, loss = 1.77700656
Validation score: 0.076233
Iteration 6, loss = 1.72395905
Validation score: 0.126689
Iteration 7, loss = 1.69841465
Validation score: 0.138775
Iteration 8, loss = 1.68449608
Validation score: 0.144251
Iteration 9, loss = 1.67637630
Validation score: 0.146969
Iteration 10, loss = 1.67266698
Validation score: 0.147224
Iteration 11, loss = 1.66796035
Validation score: 0.153735
Iteration 12, loss = 1.66375568
Validation score: 0.154154
Iteration 13, loss = 1.65824502
Validation score: 0.155965
Iteration 14, loss = 1.65656213
Validation score: 0.153967
Iteration 15, loss = 1.65294907
Validation score: 0.157080
Iteration 16, loss = 1.64863792
Validation score: 0.156553
Iteration 17, loss = 1.64790724
Validation score: 0.157286
Iteration 18, loss = 1.64560462
Validation score: 0.158154
Iteration 19, loss = 1.64360681
Validation score: 0.155185
Iteration 20, loss = 1.64318133
Validation score: 0.162404
Iteration 21, loss = 1.63902652
Validation score: 0.157453
It

# Evaluate

In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import numpy as np

print("Results for IDim – Positive")
print("______________________________________________")

# Predict and reshape if needed
y_pred_positive = mlp_regressor_positive.predict(X_test_positive)
if y_pred_positive.ndim > 1:
    y_pred_positive = y_pred_positive.flatten()
if y_test_positive.ndim > 1:
    y_test_positive = y_test_positive.flatten()

print(f"\nShape of predictions (y_pred): {y_pred_positive.shape}")
print(f"First 5 actual values (y_test):\n{y_test_positive[:5].reshape(-1, 1)}")
print(f"First 5 predicted values (y_pred):\n{y_pred_positive[:5].reshape(-1, 1)}")

print(f"Training/ Test split: {int(TRAIN_PERCENTAGE_positive * 100)}/{int(TEST_PERCENTAGE_positive * 100)}\n")

# Evaluation Metrics
mae_positive = mean_absolute_error(y_test_positive, y_pred_positive)
rmse_positive = np.sqrt(mean_squared_error(y_test_positive, y_pred_positive))
r2_positive = r2_score(y_test_positive, y_pred_positive)


print(f"Mean Absolute Error (MAE): {mae_positive:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_positive:.4f}")
print(f"R-squared (R²): {r2_positive:.4f}")

# Pearson Correlation
if np.std(y_test_positive) > 1e-6 and np.std(y_pred_positive) > 1e-6:
    corr_positive, _ = pearsonr(y_test_positive, y_pred_positive)
    print(f"Pearson Correlation (Relaxed): {corr_positive:.4f}")
else:
    print("Pearson Correlation: Cannot calculate (insufficient variance)")

#Mean Absolute Percentage Error
y_test_positive_safe = np.clip(y_test_positive, a_min=1e-6, a_max=None)
mape_positive = np.mean(np.mean(np.abs((y_test_positive - y_pred_positive) / y_test_positive_safe)) * 100)
print(f"Mean Absolute Percentage Error (MAPE): {mape_positive:.2f}%")

Results for IDim – Positive
______________________________________________

Shape of predictions (y_pred): (767,)
First 5 actual values (y_test):
[[2.01]
 [4.06]
 [1.08]
 [3.23]
 [8.02]]
First 5 predicted values (y_pred):
[[5.373487 ]
 [4.6843653]
 [3.84991  ]
 [2.9760873]
 [5.01071  ]]
Training/ Test split: 80/20

Mean Absolute Error (MAE): 1.3856
Root Mean Squared Error (RMSE): 1.7381
R-squared (R²): 0.3145
Pearson Correlation (Relaxed): 0.5635
Mean Absolute Percentage Error (MAPE): 45.99%


In [24]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import numpy as np

print("Results for IDim – Relaxed")
print("______________________________________________")

# Predict and reshape if needed
y_pred_relaxed = mlp_regressor_relaxed.predict(X_test_relaxed)
if y_pred_relaxed.ndim > 1:
    y_pred_relaxed = y_pred_relaxed.flatten()
if y_test_relaxed.ndim > 1:
    y_test_relaxed = y_test_relaxed.flatten()

print(f"\nShape of predictions (y_pred): {y_pred_relaxed.shape}")
print(f"First 5 actual values (y_test):\n{y_test_relaxed[:5].reshape(-1, 1)}")
print(f"First 5 predicted values (y_pred):\n{y_pred_relaxed[:5].reshape(-1, 1)}")

print(f"Training/ Test split: {int(TRAIN_PERCENTAGE_relaxed * 100)}/{int(TEST_PERCENTAGE_relaxed * 100)}\n")

# Evaluation Metrics
mae_relaxed = mean_absolute_error(y_test_relaxed, y_pred_relaxed)
rmse_relaxed = np.sqrt(mean_squared_error(y_test_relaxed, y_pred_relaxed))
r2_relaxed = r2_score(y_test_relaxed, y_pred_relaxed)

print(f"Mean Absolute Error (MAE): {mae_relaxed:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_relaxed:.4f}")
print(f"R-squared (R²): {r2_relaxed:.4f}")

# Pearson Correlation
if np.std(y_test_relaxed) > 1e-6 and np.std(y_pred_relaxed) > 1e-6:
    corr_relaxed, _ = pearsonr(y_test_relaxed, y_pred_relaxed)
    print(f"Pearson Correlation (Relaxed): {corr_relaxed:.4f}")
else:
    print("Pearson Correlation: Cannot calculate (insufficient variance)")

#Mean Absolute Percentage Error
y_test_relaxed_safe = np.clip(y_test_relaxed, a_min=1e-6, a_max=None)
mape_relaxed = np.mean(np.mean(np.abs((y_test_relaxed - y_pred_relaxed) / y_test_relaxed_safe)) * 100)
print(f"Mean Absolute Percentage Error (MAPE): {mape_relaxed:.2f}%")


Results for IDim – Relaxed
______________________________________________

Shape of predictions (y_pred): (767,)
First 5 actual values (y_test):
[[4.93]
 [3.06]
 [1.02]
 [1.  ]
 [8.02]]
First 5 predicted values (y_pred):
[[5.3601756]
 [4.666016 ]
 [3.2587814]
 [2.1517289]
 [4.788669 ]]
Training/ Test split: 80/20

Mean Absolute Error (MAE): 1.4343
Root Mean Squared Error (RMSE): 1.7671
R-squared (R²): 0.3342
Pearson Correlation (Relaxed): 0.5811
Mean Absolute Percentage Error (MAPE): 44.69%


In [25]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import numpy as np

print("Results for IDim – Awake")
print("______________________________________________")

# Predict and reshape if needed
y_pred_awake = mlp_regressor_awake.predict(X_test_awake)
if y_pred_awake.ndim > 1:
    y_pred_awake = y_pred_awake.flatten()
if y_test_awake.ndim > 1:
    y_test_awake = y_test_awake.flatten()

print(f"\nShape of predictions (y_pred): {y_pred_awake.shape}")
print(f"First 5 actual values (y_test):\n{y_test_awake[:5].reshape(-1, 1)}")
print(f"First 5 predicted values (y_pred):\n{y_pred_awake[:5].reshape(-1, 1)}")

print(f"Training/ Test split: {int(TRAIN_PERCENTAGE_awake * 100)}/{int(TEST_PERCENTAGE_awake * 100)}\n")

# Evaluation Metrics
mae_awake = mean_absolute_error(y_test_awake, y_pred_awake)
rmse_awake = np.sqrt(mean_squared_error(y_test_awake, y_pred_awake))
r2_awake = r2_score(y_test_awake, y_pred_awake)

print(f"Mean Absolute Error (MAE): {mae_awake:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_awake:.4f}")
print(f"R-squared (R²): {r2_awake:.4f}")

# Pearson Correlation
if np.std(y_test_awake) > 1e-6 and np.std(y_pred_awake) > 1e-6:
    corr_awake, _ = pearsonr(y_test_awake, y_pred_awake)
    print(f"Pearson Correlation (awake): {corr_awake:.4f}")
else:
    print("Pearson Correlation: Cannot calculate (insufficient variance)")

#Mean Absolute Percentage Error
y_test_awake_safe = np.clip(y_test_awake, a_min=1e-6, a_max=None)
mape_awake = np.mean(np.mean(np.abs((y_test_awake - y_pred_awake) / y_test_awake_safe)) * 100)
print(f"Mean Absolute Percentage Error (MAPE): {mape_awake:.2f}%")

Results for IDim – Awake
______________________________________________

Shape of predictions (y_pred): (767,)
First 5 actual values (y_test):
[[1.96]
 [2.92]
 [9.  ]
 [6.9 ]
 [7.03]]
First 5 predicted values (y_pred):
[[4.745537 ]
 [4.784462 ]
 [6.714811 ]
 [7.0741386]
 [5.970946 ]]
Training/ Test split: 80/20

Mean Absolute Error (MAE): 1.4946
Root Mean Squared Error (RMSE): 1.9015
R-squared (R²): 0.0578
Pearson Correlation (awake): 0.2516
Mean Absolute Percentage Error (MAPE): 39.77%


In [26]:
# Organize metrics into a dictionary
print("Results for Dimensional Induced (IDim)")
print("______________________________________________")
data = {
    "Dimension": ["Positive", "Relaxed", "Awake"],
    "MAE": [mae_positive, mae_relaxed, mae_awake],
    "RMSE": [rmse_positive, rmse_relaxed, rmse_awake],
    "R²": [r2_positive, r2_relaxed, r2_awake],
    "Pearson Corr.": [corr_positive, corr_relaxed, corr_awake],
    "MAPE": [mape_positive, mape_relaxed, mape_awake]
}

# Create DataFrame
df_metrics = pd.DataFrame(data)

# Add average row
avg_row = {
    "Dimension": "Average",
    "MAE": df_metrics["MAE"].mean(),
    "RMSE": df_metrics["RMSE"].mean(),
    "R²": df_metrics["R²"].mean(),
    "Pearson Corr.": df_metrics["Pearson Corr."].mean(),
    "MAPE": df_metrics["MAPE"].mean()
}
df_metrics = pd.concat([df_metrics, pd.DataFrame([avg_row])], ignore_index=True)


# Display
print(df_metrics.round(4))


Results for Dimensional Induced (IDim)
______________________________________________
  Dimension     MAE    RMSE      R²  Pearson Corr.     MAPE
0  Positive  1.3856  1.7381  0.3145         0.5635  45.9882
1   Relaxed  1.4343  1.7671  0.3342         0.5811  44.6924
2     Awake  1.4946  1.9015  0.0578         0.2516  39.7662
3   Average  1.4382  1.8022  0.2355         0.4654  43.4823
